In [24]:
# Import Python libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval

In [25]:
data = pd.read_csv('StackOverflow_cleaned.csv',sep=";", index_col=0,converters={"Title": literal_eval,
                                                                                 "Body": literal_eval,
                                                                                  "Tags": literal_eval})

In [26]:
data=data[0:100]

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
X = data["Body"]
y = data["Tags"]

vectorizer = TfidfVectorizer(analyzer="word",
                             max_df=.6,
                             min_df=0.005,
                             tokenizer=None,
                             preprocessor=' '.join,
                             stop_words=None,
                             lowercase=False)


vectorizer.fit(X)
X_tfidf = vectorizer.transform(X)


In [28]:
X_tfidf.shape

(100, 791)

In [29]:
X_tfidf

<100x791 sparse matrix of type '<class 'numpy.float64'>'
	with 1651 stored elements in Compressed Sparse Row format>

RandomForest ONeVsRest

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split



# transform output : 
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(y)
y_bin = multilabel_binarizer.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_bin, test_size=0.3, random_state=8)



In [31]:
!pip install nested-cv

In [34]:
from nested_cv import NestedCV
from sklearn.metrics import f1_score
# Initialize RandomForest with OneVsRest
param_rfc = {"estimator__max_depth": [5, 25, 50],
             "estimator__min_samples_leaf": [1, 5, 10],
             "estimator__class_weight": ["balanced"]}


multi_rfc_cv = NestedCV(model=OneVsRestClassifier(RandomForestClassifier()) , params_grid=param_rfc,
               outer_kfolds=5, inner_kfolds=5, n_jobs = -1,
               cv_options={'metric':f1_score,
                            'metric_score_indicator_lower' : False,
                            #'randomized_search_iter':30, 
                            'multiclass_average' : "weighted",
                           #'rfe_n_features':2
                           })
'''
multi_rfc_cv = GridSearchCV(OneVsRestClassifier(RandomForestClassifier()),
                            param_grid=param_rfc,
                            n_jobs=-1,
                            cv=2,
                            scoring="f1_weighted",
                            return_train_score = True,
                            refit=True,
                            verbose=3)'''
# Fit on Sample data
multi_rfc_cv.fit(X_train, y_train)

C:\Users\Aeorn\anaconda3\lib\site-packages\sklearn\multiclass.py:79: UserWarning: Label not 16 is present in all training examples.
  warnings.warn(
C:\Users\Aeorn\anaconda3\lib\site-packages\sklearn\multiclass.py:79: UserWarning: Label not 25 is present in all training examples.
  warnings.warn(
C:\Users\Aeorn\anaconda3\lib\site-packages\sklearn\multiclass.py:79: UserWarning: Label not 33 is present in all training examples.
  warnings.warn(
C:\Users\Aeorn\anaconda3\lib\site-packages\sklearn\multiclass.py:79: UserWarning: Label not 40 is present in all training examples.
  warnings.warn(
C:\Users\Aeorn\anaconda3\lib\site-packages\sklearn\multiclass.py:79: UserWarning: Label not 44 is present in all training examples.
  warnings.warn(
C:\Users\Aeorn\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
 

In [36]:
#rfc_cv_results = pd.DataFrame.from_dict(multi_rfc_cv.cv_results_)
#rfc_best_params = multi_rfc_cv.best_params_
#print(rfc_best_params)

AttributeError: 'NestedCV' object has no attribute 'best_params_'

In [ ]:
#rfc_best_params_ok = {}
#for k, v in rfc_best_params.items():
 #   rfc_best_params_ok[k.replace("estimator__","")] = v

In [37]:
# Refit RandomForestClassifier best_params with full dataset
rfc_final_model = multi_rfc_cv#OneVsRestClassifier(RandomForestClassifier(**rfc_best_params_ok))
#rfc_final_model.fit(X_train, y_train)

# Predict
y_test_predicted_labels_tfidf_rfc = rfc_final_model.predict(X_test)

# Inverse transform
y_test_pred_inversed_rfc = multilabel_binarizer.inverse_transform(y_test_predicted_labels_tfidf_rfc)
y_test_inversed = multilabel_binarizer.inverse_transform(y_test)
print("Predicted:", y_test_pred_inversed_rfc[0:5])
print("True:", y_test_inversed[0:5])

AttributeError: 'NestedCV' object has no attribute 'predict'

In [ ]:
def metrics_score(model, df, y_true, y_pred):
 
    if(df is not None):
        temp_df = df
    else:
        temp_df = pd.DataFrame(index=["Accuracy", "F1",
                                      "Jaccard", "Recall",
                                      "Precision"],
                               columns=[model])
        
    scores = []
    scores.append(metrics.accuracy_score(y_true, 
                                         y_pred))
    scores.append(metrics.f1_score(y_pred, 
                                   y_true, 
                                   average='weighted'))
    scores.append(metrics.jaccard_score(y_true, 
                                        y_pred, 
                                        average='weighted'))
    scores.append(metrics.recall_score(y_true, 
                                       y_pred, 
                                       average='weighted'))
    scores.append(metrics.precision_score(y_true, 
                                          y_pred, 
                                          average='weighted'))
    temp_df[model] = scores
    
    return temp_df
    
df_metrics_compare = metrics_score("RandomForest", 
                                   df=None,
                                   y_true = y_test,
                                   y_pred = y_test_predicted_labels_tfidf_rfc)
df_metrics_compare


,RandomForest
Accuracy,0.166482
F1,0.529583
Jaccard,0.390206
Recall,0.626744
Precision,0.501085
